In [1]:
#!pip install numpy

In [2]:
#!pip install flask

In [5]:
#!pip install mtcnn-0.1.0-py3-none-any.whl

Processing c:\users\sm-02\파이썬\3차프로젝트\mtcnn-0.1.0-py3-none-any.whl


In [4]:
#!pip install flask-mysqldb

In [5]:
#!pip install tqdm

In [6]:
# conda install -c pytorch pytorch   (아나콘다 프롬프트 창에서)
# conda install ytorchvision -c pytorch

In [1]:
import pandas as pd
from flask import Flask, render_template, request, redirect, url_for, session, flash, jsonify
from werkzeug.utils import secure_filename
#from werkzeug.datastructures import FileStorage
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
from flask_mysqldb import MySQL

Using TensorFlow backend.


In [2]:
#!pip install flask-mysqldb

In [3]:
from tqdm import tqdm_notebook
from collections import Counter

In [4]:
import os, sys, time
import cv2
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline
import matplotlib.pyplot as plt

test_dir = "./static/testMp4/"
test_videos = sorted([x for x in os.listdir(test_dir) if x[-4:] == ".mp4"])
frame_h = 5
frame_l = 5
len(test_videos)

gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gpu

import sys
sys.path.insert(0, "./blazeface-pytorch")
sys.path.insert(0, "./deepfakes-inference-demo")

from blazeface import BlazeFace
facedet = BlazeFace().to(gpu)
facedet.load_weights("./blazeface-pytorch/blazeface.pth")
facedet.load_anchors("./blazeface-pytorch/anchors.npy")
_ = facedet.train(False)

from helpers.read_video_1 import VideoReader
from helpers.face_extract_1 import FaceExtractor

frames_per_video = 64
video_reader = VideoReader()
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

input_size = 224

from torchvision.transforms import Normalize

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize_transform = Normalize(mean, std)

def isotropically_resize_image(img, size, resample=cv2.INTER_AREA):
    h, w = img.shape[:2]
    if w > h:
        h = h * size // w
        w = size
    else:
        w = w * size // h
        h = size

    resized = cv2.resize(img, (w, h), interpolation=resample)
    return resized


def make_square_image(img):
    h, w = img.shape[:2]
    size = max(h, w)
    t = 0
    b = size - h
    l = 0
    r = size - w
    return cv2.copyMakeBorder(img, t, b, l, r, cv2.BORDER_CONSTANT, value=0)

import torch.nn as nn
import torchvision.models as models

class MyResNeXt(models.resnet.ResNet):
    def __init__(self, training=True):
        super(MyResNeXt, self).__init__(block=models.resnet.Bottleneck,
                                        layers=[3, 4, 6, 3], 
                                        groups=32, 
                                        width_per_group=4)
        self.fc = nn.Linear(2048, 1)
        
checkpoint = torch.load("./checkpoint.pth", map_location=gpu)

model = MyResNeXt().to(gpu)
model.load_state_dict(checkpoint)
_ = model.eval()

del checkpoint

def predict_on_video(video_path, batch_size):
    try:
        # Find the faces for N frames in the video.
        faces = face_extractor.process_video(video_path)

        # Only look at one face per frame.
        face_extractor.keep_only_best_face(faces)
        
        if len(faces) > 0:
            # NOTE: When running on the CPU, the batch size must be fixed
            # or else memory usage will blow up. (Bug in PyTorch?)
            x = np.zeros((batch_size, input_size, input_size, 3), dtype=np.uint8)

            # If we found any faces, prepare them for the model.
            n = 0
            for frame_data in faces:
                for face in frame_data["faces"]:
                    # Resize to the model's required input size.
                    # We keep the aspect ratio intact and add zero
                    # padding if necessary.                    
                    resized_face = isotropically_resize_image(face, input_size)
                    resized_face = make_square_image(resized_face)

                    if n < batch_size:
                        x[n] = resized_face
                        n += 1
                    else:
                        print("WARNING: have %d faces but batch size is %d" % (n, batch_size))
                    
                    # Test time augmentation: horizontal flips.
                    # TODO: not sure yet if this helps or not
                    #x[n] = cv2.flip(resized_face, 1)
                    #n += 1

            if n > 0:
                x = torch.tensor(x, device=gpu).float()

                # Preprocess the images.
                x = x.permute((0, 3, 1, 2))

                for i in range(len(x)):
                    x[i] = normalize_transform(x[i] / 255.)

                # Make a prediction, then take the average.
                with torch.no_grad():
                    y_pred = model(x)
                    y_pred = torch.sigmoid(y_pred.squeeze())
                    return y_pred[:n].mean().item()

    except Exception as e:
        print("Prediction error on video %s: %s" % (video_path, str(e)))

    return 0.5

from concurrent.futures import ThreadPoolExecutor

In [ ]:
app = Flask(__name__)
#model = pickle.load(open('model.pkl','rb'))
app.secret_key = 'xJ1\xd1\xc4\xcf\xed2C\xfc\x95\x92'

# db connect
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = '1234'
app.config['MYSQL_DB'] = 'dbdeep'

mysql = MySQL(app)

@app.route("/")
def home():
    # db 연결
    cur = mysql.connection.cursor()
    cur.execute("select * from video")
    rows = cur.fetchall()
    cur.close()
    
    tag_list = []
    for obj in rows :
        tag = obj[2]
        tag_list.append(tag)
        
    tag_result = Counter(tag_list)
    result_list = tag_result.most_common(10)
    
    tag_lists = []
    for obj in result_list :
        tag = obj[0]        # tag_list
        tag_lists.append(tag)
        
    #print(tag_lists)
    return render_template('main.html', tag_lists = tag_lists)

@app.route("/nav")
def nav(): 
    return render_template('nav.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":

        id = request.form['id']
        password = request.form['password']

        cur = mysql.connection.cursor()
        cur.execute("SELECT id, name, password, age, sex FROM user WHERE id = %s AND password = %s", (id, password))
        
        account = cur.fetchone()
        
        if account:
            session['login'] = True
            session['id'] = account[0]
            session['name'] = account[1]
            session['password'] = account[2]
            session['age'] = account[3]
            session['sex'] = account[4]

            return jsonify({"status": "success"}) # json형태로 상태를 리턴 해줌 
        else:
            return jsonify({"status": "fail"})
    return render_template('login.html')


@app.route('/logout')
def logout():
    session.pop('login', None)
    session.pop('id', None)
    session.pop('name', None)
    session.pop('password', None)
    session.pop('age', None)
    session.pop('sex', None)
    
    return redirect(url_for('home'))

@app.route('/join', methods=['GET', 'POST'])
def join():
    if request.method == "POST":

        id = request.form['id']
        name = request.form['name']
        password = request.form['password']
        password_check = request.form['password_check']
        age = request.form['age']
        sex = request.form['sex']

        if(password == password_check):
            cur = mysql.connection.cursor()
            cur.execute("INSERT INTO user(id, name, password, age, sex) VALUES (%s, %s, %s, %s, %s)", (id, name, password, age, sex))
            mysql.connection.commit()
            cur.close()
            return jsonify({"status": "success"})
        else:
            return jsonify({"status": "fail"})

    return render_template('join.html')


@app.route('/mypage', methods=['GET', 'POST'])
def mypage():
    id = session['id']
    print("mypage id:",id)
    # db 연결
    cur = mysql.connection.cursor()
    cur.execute("select * from video where id = '%s'" % id)
    video = cur.fetchall()
    cur.close()
    
    #이용내역
    video_list = []
    result_list = []
    for obj in video :
        video_name = obj[1] # 파일이름
        result = obj[3]     #결과
        
        video_list.append(video_name)
        result_list.append(result)
   
        
    #문의 내역
    cur = mysql.connection.cursor()
    cur.execute("select * from board where id = '%s' order by num desc" % id)

    list = cur.fetchall()
        
    
    # edit_user    
    if request.method == "POST":
        name = request.form['name']
        password = request.form['password']
        password_check = request.form['password_check']
        

        if password == password_check:
            if name:
                name = name
                session['name'] = name
            else:
                name = session['name']

            if password:
                password = password
                session['password'] = password
            else:
                password = session['password']

            
            cur = mysql.connection.cursor()
            cur.execute("UPDATE user SET name=%s, password=%s WHERE id = %s", (name, password, id))
            mysql.connection.commit()
            cur.close()
            return jsonify({"status": "success"})
        
        else:
            return jsonify({"status": "fail"})

    return render_template('mypage.html', video_list = video_list, result_list = result_list, list = list)

@app.route("/manual")
def manual():                           
    return render_template('manual.html')


@app.route("/upload")
def upload():                           
    return render_template('upload.html')


@app.route("/fileUpload", methods = ['GET', 'POST'])
def upload_file():         
    if request.method == 'POST':
        
        f = request.files
        print("1",f)
        print("2",f['files'])
        print("3",f['files'].filename)
        f['files'].save("static/testMp4/" + secure_filename(f['files'].filename))
        filenames = f['files'].filename
        
        def predict_on_video_set(videos, num_workers):
            filename = filenames
            y_pred = predict_on_video(os.path.join(test_dir, filename), batch_size=frames_per_video)
            return y_pred

            with ThreadPoolExecutor(max_workers=num_workers) as ex:
                predictions = ex.map(process_file, range(len(videos)))

            return list(predictions)

        preds = predict_on_video_set(filenames, num_workers=4)
        print(preds)
        
        # 진짜/가짜 t_f
        if(preds >=0.50 and preds <=1):
            t_f = '가짜영상'
        else :
            t_f = '진짜영상'
        
        # 진짜/가짜 percent
        if(preds<=0.09):
            percent = "99%"
        elif(preds<=0.19):
            percent = "90%"
        elif(preds <= 0.29):
            percent = "80%"
        elif(preds <= 0.39):
            percent = "70%"
        elif(preds <= 0.49):
            percent = "60%"
        elif(preds <= 0.59):
            percent = "50%"
        elif(preds <= 0.69):
            percent = "60%"
        elif(preds <= 0.79):
            percent = "70%"
        elif(preds <= 0.89):
            percent = "80%"
        elif(preds <= 0.95):
            percent = "90%"
        elif(preds <= 1):
            percent = "99%"

        print(percent)
        
        #db 연결
        # tag, source
        if session['login']:
            id = session['id']
        tag = request.form['tag']
        source = request.form['source']
        #print(tag)
        #print(source)
        
        # db 연결
        cur = mysql.connection.cursor()
        cur.execute("INSERT INTO video VALUES (%s, %s, %s, %s, now(), %s)", (id, filenames, tag, t_f, source))
        mysql.connection.commit()
        cur.close()


    return render_template('result.html', result = filenames, predict = percent, t_f = t_f)

@app.route("/result")
def result(result):
    print("result : ",result)
    return render_template('result.html', result = result, predict = preds, f = f)

@app.route("/stat")
def stat():
    # db 연결
    cur = mysql.connection.cursor()
    cur.execute("select * from video")
    rows = cur.fetchall()
    
   # rows data_list에 넣기
    data_list = []
    source_list = []
    tag_list = []

    for obj in rows :
        data = obj[3]         # 가짜/진짜
        source = obj[5]       # 출처
        tag = obj[2]
        
        data_list.append(data)
        source_list.append(source)
        tag_list.append(tag)
        
     # 원형 차트(가짜/진짜)    
    #print(data_list)   #label
    #print(len(data_list))  #count
    fake = data_list.count('가짜영상')
    real = data_list.count('진짜영상')
    count = [fake,real]
    
    # 출처 차트
    youtube = source_list.count('유튜브')
    naver = source_list.count('네이버')
    google = source_list.count('구글')
    daum = source_list.count('다음')
    etc = source_list.count('기타')
    source_count = [youtube, naver, google, daum, etc]
    
    # 태그 차트
    #print(tag_list)
    tag_result = Counter(tag_list)
    result_list = tag_result.most_common(10)
    tag_lists = []
    tag_count_lists = []
    for obj in result_list :
        tag = obj[0]        # tag_list
        tag_count = obj[1]  # tag_count
        tag_lists.append(tag)
        tag_count_lists.append(tag_count)
    
    # 나이대별 차트
    cur.execute("select * from user")
    user = cur.fetchall()
    age_list = []

    for obj in user :
        age = obj[3]         # 나이
        age_list.append(age)
    
    age_10 = age_list.count(10)
    age_20 = age_list.count(20)
    age_30 = age_list.count(30)
    age_40 = age_list.count(40)
    age_50 = age_list.count(50)
    age_60 = age_list.count(60)
    age_count = [age_10, age_20, age_30, age_40, age_50, age_60]
    
    
    return render_template('stat.html', result = data_list, count = count, source_count = source_count, tag_lists = tag_lists,
                          tag_count_lists = tag_count_lists, age_count = age_count)

@app.route("/intro")
def intro():
    return render_template('intro.html')

@app.route('/write', methods=['GET', 'POST'])
def write():
    if request.method == "POST":

        title = request.form['title']
        id = session['id']
        content = request.form['content']
        
        try:
            cur = mysql.connection.cursor()
            cur.execute("INSERT INTO board(title, id, content) VALUES (%s, %s, %s)", (title, id, content))
            mysql.connection.commit()
            cur.close()
            return jsonify({"status": "success"})
        except:
            return jsonify({"status": "fail"})

    return render_template('board_write.html')


@app.route('/list', methods=['GET'])
def board_list():
    import math
    cur = mysql.connection.cursor()
    cur.execute("SELECT COUNT(*) FROM board")
    total_count = cur.fetchone()
    cur.close()        
    page = math.ceil(int(total_count[0])/5)
        
    if request.args.get('page') != None:
        current_page = int(request.args.get('page'))
    else:
        current_page = 1
        
    limit_page = (current_page - 1) * 5
        
    list_query = "SELECT * FROM board ORDER BY num DESC LIMIT %d, 5" % limit_page
        
    cur2 = mysql.connection.cursor()
    cur2.execute(list_query)
    list = cur2.fetchall()
    cur2.close()
      
    return render_template("board.html",list = list, page = int(page))


@app.route('/board_detail', methods=['GET'])
def board_detail():
    num = request.args.get('num')
    
    cur = mysql.connection.cursor()
    cur.execute("SELECT * FROM board WHERE num = %s", (num))
    detail = cur.fetchall()
    cur.close()
    
    return render_template("board_detail.html",detail = detail)

if __name__ == "__main__":
    app.run(host="localhost", port="9003", threaded=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9003/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2020 14:22:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:22:57] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:22:57] "GET /static/css/fontawesome-all.min.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:22:58] "GET /static/js/scroll.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:22:58] "GET /static/js/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:22:58] "GET /static/images/phone.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:22:58] "GET /static/webfonts/fa-solid-900.woff2 HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:22:58] "GET /static/js/jquery.dropotron.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:22:58] "GET /static/js/browser.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:22:58] "GET /static/js/breakpoints.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:22:58] "GET /static/js/util.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020

mypage id: ayoung


127.0.0.1 - - [14/Mar/2020 14:27:05] "GET /mypage HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:27:06] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:27:06] "GET /static/css/images/video.JPG HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2020 14:27:06] "GET /static/testMp4/realObama.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:27:06] "GET /static/testMp4/jin_real.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:27:06] "GET /static/testMp4/realObama.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [14/Mar/2020 14:27:06] "GET /static/testMp4/jin_real.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [14/Mar/2020 14:27:06] "GET /static/testMp4/realObama.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [14/Mar/2020 14:27:06] "GET /static/testMp4/jin_real.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [14/Mar/2020 14:27:07] "GET /static/testMp4/realObama.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [14/Mar/2020 14:27:07] "GET /static/testMp4/jin_real.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [14/Mar/2020 14:27:07] "GET /static/testMp4/realObama.mp4 HTTP/1.1" 20

1 ImmutableMultiDict([('files', <FileStorage: 'jaesuk.mp4' ('video/mp4')>)])
2 <FileStorage: 'jaesuk.mp4' ('video/mp4')>
3 jaesuk.mp4
0.01690565049648285
99%


...

127.0.0.1 - - [14/Mar/2020 14:40:12] "POST /fileUpload HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:40:12] "GET /static/testMp4/obama2.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:40:13] "GET /static/testMp4/obama2.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:40:13] "GET /static/testMp4/obama2.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [14/Mar/2020 14:40:13] "GET /static/testMp4/obama2.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [14/Mar/2020 14:40:13] "GET /stat HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:40:14] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:40:24] "GET /upload HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:40:24] "GET /nav HTTP/1.1" 200 -


1 ImmutableMultiDict([('files', <FileStorage: 'jaesuk.mp4' ('video/mp4')>)])
2 <FileStorage: 'jaesuk.mp4' ('video/mp4')>
3 jaesuk.mp4
0.01690565049648285
99%


127.0.0.1 - - [14/Mar/2020 14:40:42] "POST /fileUpload HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:40:44] "GET /static/testMp4/jaesuk.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:40:44] "GET /static/testMp4/jaesuk.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:40:45] "GET /static/testMp4/jaesuk.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [14/Mar/2020 14:40:45] "GET /static/testMp4/jaesuk.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [14/Mar/2020 14:40:45] "GET /stat HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:40:46] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:41:11] "GET /stat HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:41:11] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:41:21] "GET /upload HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:41:21] "GET /nav HTTP/1.1" 200 -


1 ImmutableMultiDict([('files', <FileStorage: 'youngsun.mp4' ('video/mp4')>)])
2 <FileStorage: 'youngsun.mp4' ('video/mp4')>
3 youngsun.mp4
0.6358108520507812
60%


127.0.0.1 - - [14/Mar/2020 14:41:36] "POST /fileUpload HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:41:36] "GET /stat HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:41:37] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:41:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:41:44] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:41:47] "GET /static/css/images/navPanel.svg HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:41:54] "GET /main.html HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2020 14:42:05] "GET /stat HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:42:06] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:42:11] "GET /stat HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:42:11] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:42:20] "GET /stat HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2020 14:42:20] "GET /nav HTTP/1.1" 200 -


In [6]:
# filenames = "realYoung2.mp4"
# def predict_on_video_set(videos, num_workers):
#     filename = filenames
#     y_pred = predict_on_video(os.path.join(test_dir, filename), batch_size=frames_per_video)
    
#     return y_pred

#     with ThreadPoolExecutor(max_workers=num_workers) as ex:
#         predictions = ex.map(process_file, range(len(videos)))

#     return list(predictions)

# preds = predict_on_video_set(filenames, num_workers=4)
# print(preds)

0.4822807312011719
